# Test to define how to add decision var with multiple sets Var(x,t) into a dataframe to connect to Machine Learning Model

Gurobi machine learning, how is natural, only accepts dataframe of 2 dimensions to connect to Machine Learning Models. **The columns are the features that the model recibe (always are the features of the ml model) and the rows represent one set of the data**, 

So, when the decision variables has multiple sets there are two aproaches that can be do depending of the nature of the optimization problem:
- Use a ML model transversal across only one of the sets (time for example) to achieve to generate the input dataframe: time, features
- Use a ML model with the input dataframe with multiindex to have a input dataframe with 2 or more sets: (product, time), feartures

## Root folder and read env variables

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-2]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## ---> LOAD THE PROCESS THAT PREDICT OUTPUT Y2 OF PROCESS B.  Y2 = f(Z1, X2, O5, O6)

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

### 0. Load data
This data will be use to get values to generate a instance of the ml model

In [ ]:
name_process = 'process_b_y2'  # LOAD THE MODEL THAT PREDICT OUTPUT Y2 OF PROCESS B.  Y2 = f(Z1, X2, O5, O6)

# load X_test
path_X_test = f'artifacts/data_training/{name_process}/X_test.pkl'
X_test = pd.read_pickle(path_X_test)

# load y_test
path_y_test = f'artifacts/data_training/{name_process}/y_test.pkl'
y_test = pd.read_pickle(path_y_test)

### 1. Load Artifacts to connect ML to gurobi

#### 1.1 pkl model

In [ ]:
path_model_to_test = f'artifacts/models/{name_process}/lr.pkl'
model_ml_to_test = pd.read_pickle(path_model_to_test)
model_ml_to_test

### 1.2 Define list of features and target for each model

In [ ]:
X_test

In [ ]:
######################## model  ########################

list_features = ['Z1', 'X2', 'O5', 'O6']

list_features_controlables = ['Z1', 'X2']

list_target = ['Y2']

### 1.3 Read master tag and sort features according its order

In [ ]:
# read table master tag
path_list_features_target_to_optimization = f'config/config_ml_models_development/MasterTable_{name_process}.xlsx'
maestro_tags = pd.read_excel(path_list_features_target_to_optimization)

### sort list of features according the order in master table
list_features = [tag for tag in maestro_tags['TAG'].tolist() if tag in list_features]
list_features_controlables = [tag for tag in maestro_tags['TAG'].tolist() if tag in list_features_controlables]

## 2. Create gurobi model

In [ ]:
# create model
m = gp.Model('modelo')

### 3. Create decision variables
- Decision variables that are features in ml models
- Decicion variable that is the output in ml models

#### 3.1 define multiple sets

In [ ]:
# define set time
list_set_time = ['t0', 't1', 't2', 't3', 't4', 't5', 't6']
index_set_time = pd.Index(list_set_time)
index_set_time

In [ ]:
# define set additional - second set defined to this example - of example, think the set represent the kind of product
list_set_product = ['w', 'x', 'y', 'z']
index_set_product = pd.Index(list_set_product)
index_set_product

In [ ]:
############## multi set region&time - index pandas ##############
index_set_product_time = pd.MultiIndex.from_product((list_set_product, list_set_time), 
                                                    names = ('product', 'time')
                                                   )
index_set_product_time

#### 3.2 create decision variables MULTIPLE INDEX

In [ ]:
# create decision variables - features ml model
var_Z1 = gppd.add_vars(m, index_set_product_time, name = "decision variable Z1"
                                     )

var_X2 = gppd.add_vars(m, index_set_product_time, name = "decision variable X2"
                                     )

In [ ]:
# crete decision variables - output ml model
var_Y2 = gppd.add_vars(m, index_set_product_time, name = "decision variable Y2"
                                     )

In [ ]:
# "compile"
m.update()

In [ ]:
# see decision var created
var_Y2

## EXAMPLES ML MODELS WITH DECISION VAR WITH MULTIPLE SETS

### A. Instance Machine learning MULTIPLE INDEX
- Create instance of Machine learning model using decision var of gurobi (decision var in optimization)

- The observed variables has fixed values, so this values doesn't change across the time

In [ ]:
######################## generate instance NO controlables features for model ########################

# list feature NC
list_features_no_vc = list(set(list_features) - set(list_features_controlables))

# generate dataframe with input values. In this example is the mean value
df_input_values = np.array(X_test[list_features_no_vc].mean().to_frame().T).tolist()

# generate dataframe instance_no_controlables with the time set - MULTIINDEX
instance_no_controlables = pd.DataFrame(df_input_values, index = index_set_product_time, columns = list_features_no_vc) # TODO: full index dataframe - multiindex
instance_no_controlables

In [ ]:
######################## genrate instance - features no controlables + decision vars ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables = pd.DataFrame([var_Z1, var_X2]).T # ADD DECISION VARIABLES
instance_controlables.columns = list_features_controlables # rename columns

# append features controlables with no controlables
instance = pd.concat([instance_no_controlables, instance_controlables], axis = 1)
instance = instance[list_features] # sort features

In [ ]:
instance

In [ ]:
###### load ml constraint ######
pred_constr = add_predictor_constr(gp_model = m, 
                                                    predictor = model_ml_to_test, 
                                                    input_vars = instance, # instance pandas gurobi
                                                    output_vars = var_Y2, # target
                                                    name = f'model_predict'
                                                   )
pred_constr.print_stats()

### B. Instance Machine learning ONE INDEX
- Create instance of Machine learning model using decision var of gurobi (decision var in optimization)

- The observed variables has fixed values, so this values doesn't change across the time

- For this example, supose that in the set "product" the element "w" was selected, **so the machine learning model predict for the "product w" across all the "time t"**

In [ ]:
######################## generate instance NO controlables features for model ########################

# list feature NC
list_features_no_vc = list(set(list_features) - set(list_features_controlables))

# generate dataframe with input values. In this example is the mean value
#df_input_values = X_test[list_features_no_vc].mean().to_frame().T
df_input_values = np.array(X_test[list_features_no_vc].mean().to_frame().T).tolist()

# generate dataframe instance_no_controlables with the time set - ONLY ONE INDEX
instance_no_controlables = pd.DataFrame(df_input_values, index = index_set_time, columns = list_features_no_vc) # index dataframe - multiindex
instance_no_controlables

In [ ]:
######################## genrate instance - features no controlables + decision vars ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables = pd.DataFrame([var_Z1['w'], var_X2['w']]).T # TODO: ADD DECISION VARIABLES WITH THE SELECTION OF THE SET
instance_controlables.columns = list_features_controlables # rename columns

# append features controlables with no controlables
instance = pd.concat([instance_no_controlables, instance_controlables], axis = 1)
instance = instance[list_features] # sort features

In [ ]:
instance

In [ ]:
###### load ml constraint ######
pred_constr = add_predictor_constr(gp_model = m, 
                                                    predictor = model_ml_to_test, 
                                                    input_vars = instance, # instance pandas gurobi
                                                    output_vars = var_Y2['w'], # TODO: ADD DECISION VARIABLES WITH THE SELECTION OF THE SET
                                                    name = f'model_predict'
                                                   )
pred_constr.print_stats()

### 5. Define objective optimization
Objetive that no generate infeasibility

In [ ]:
var_Y2.sum() # sum across time

In [ ]:
m.setObjective(var_Y2.sum(),
               gp.GRB.MINIMIZE)

#### 6. Optimize and get optimal values

In [ ]:
# solve
m.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

######################## save optimal values - features of models (only the features) ########################

# model
solution["var_Z1"] = var_Z1.gppd.X
solution["var_X2"] = var_X2.gppd.X


######################## save optimal values - targets of models (some targets are features of the model of the next step) ########################
solution["var_Y2"] = var_Y2.gppd.X  # model


######################## # get value objetive function ########################
opt_objetive_function = m.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution